In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version="123.0.6312.105").install()))


In [2]:
driver.get("https://www.koreabaseball.com/")


In [3]:
# from selenium.webdriver.common.by import By #driver에게 찾아줘 css selector 카피했는데 너가클릭해줘)
#lnb > li:nth-child(3) > a
from selenium.webdriver.common.by import By
driver.find_element(By.CSS_SELECTOR, "#lnb > li:nth-child(3) > a").click()

#cphContents_cphContents_cphContents_ddlTeam > option:nth-child(2) 
#cphContents_cphContents_cphContents_ddlTeam > option:nth-child(3)
#cphContents_cphContents_cphContents_ddlTeam > option:nth-child(11)
#나중에 페이지 로딩걸리면 시간조율해주는 네트워크함수알아야할듯.

#driver.find_element(By.CSS_SELECTOR, "#cphContents_cphContents_cphContents_ddlTeam > option:nth-child(2) ").click()


In [8]:
driver.page_source #자바스크립트 해석이 된 후에

'<html lang="ko"><head><style type="text/css" class="bc-style-vjs" data-vjs-version="8.11.5">.vjs-svg-icon{display:inline-block;background-repeat:no-repeat;background-position:center;fill:currentColor;height:1.8em;width:1.8em}.vjs-svg-icon:before{content:none!important}.vjs-control:focus .vjs-svg-icon,.vjs-svg-icon:hover{filter:drop-shadow(0 0 .25em #fff)}.video-js .vjs-big-play-button .vjs-icon-placeholder:before,.video-js .vjs-modal-dialog,.vjs-button>.vjs-icon-placeholder:before,.vjs-modal-dialog .vjs-modal-dialog-content{position:absolute;top:0;left:0;width:100%;height:100%}.video-js .vjs-big-play-button .vjs-icon-placeholder:before,.vjs-button>.vjs-icon-placeholder:before{text-align:center}@font-face{font-family:VideoJS;src:url(data:application/font-woff;charset=utf-8;base64,d09GRgABAAAAABUgAAsAAAAAItAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAABHU1VCAAABCAAAADsAAABUIIslek9TLzIAAAFEAAAAPgAAAFZRiV33Y21hcAAAAYQAAAEJAAAD5p42+VxnbHlmAAACkAAADwwAABdk9R/WHmhlYWQAABGcAAAAKwAAADYn8kSnaGhlYQAAEcgAAAAdA

In [13]:
from tqdm import tqdm

In [14]:
import time
import re
#1주일에 45시간 900시
pattern = re.compile("playerId=([0-9]+)")
#cphContents_cphContents_cphContents_ucPager_btnNo2
#cphContents_cphContents_cphContents_ddlTeam
#cphContents_cphContents_cphContents_ddlTeam
#cphContents_cphContents_cphContents_ddlTeam
select_page = "#cphContents_cphContents_cphContents_ucPager_btnNo{}"
select_team = "#cphContents_cphContents_cphContents_ddlTeam > option:nth-child({})"
playid = []
for x in tqdm(range(2,12)):
    for_1 = select_team.format(x)
    driver.find_element(By.CSS_SELECTOR, for_1).click()
    time.sleep(2)
    #playid.extend(pattern.findall(driver.page_source))
    for y in range(1,6):
        f2 = select_page.format(y)
        try:
            driver.find_element(By.CSS_SELECTOR, f2).click()
            time.sleep(1)
            playid.extend(pattern.findall(driver.page_source))
        except Exception as e:
            print ("page 없음 ")
        time.sleep(2)


 40%|█████████████████████████████████▏                                                 | 4/10 [01:11<01:47, 17.89s/it]

page 없음 


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:58<00:00, 17.81s/it]


In [15]:
len(list(set(playid)))


844

In [16]:
print(playid)

['50705', '52769', '54707', '54708', '71837', '69702', '51713', '51715', '54705', '68707', '65764', '65769', '50710', '53754', '54709', '50711', '66715', '63765', '52706', '68701', '66704', '69518', '50720', '69737', '76715', '52701', '51705', '53764', '66707', '67703', '68703', '51761', '69974', '53714', '54704', '53711', '53704', '51767', '54701', '53717', '51704', '79608', '52702', '69703', '51202', '54702', '69766', '52765', '63464', '54703', '65703', '76849', '65616', '53795', '65117', '52704', '52714', '68706', '68797', '54706', '68700', '67993', '76812', '66657', '63749', '60768', '50704', '51762', '79764', '77318', '69645', '66706', '54794', '74857', '68743', '69767', '54768', '51796', '69705', '65707', '79192', '50707', '78288', '54710', '52720', '54730', '62700', '52710', '50726', '61666', '69706', '54795', '54729', '54607', '54613', '67604', '68069', '61353', '53609', '51640', '76225', '66614', '69620', '60337', '52605', '53618', '69745', '54614', '69627', '54608', '54645', 

In [18]:
#abc에 모든 선수들의 번호가 저장됨
# for i in range(len(abc)):
#팀이름 밑에 집계 리스트 844개 안에 딕셔너리로
from bs4 import BeautifulSoup as BS 
import requests
select_user="https://www.koreabaseball.com/Record/Player/PitcherDetail/Basic.aspx?playerId={}"
user_info=[] # list선언 이지만 안에는 dict
user_dict={}
for i in tqdm(range(len(list(set(playid))))):
    r = requests.get(select_user.format(playid[i])) #headers부분 안적어도되는듯
    bs=BS(r.text)
    title = bs.find("div", class_="player_info").text.strip()
    list1=title.split("\n")
    #print(str(list1))
    user_dict={}
    user_dict["구단"]=list1[0]
    #dict(a)

    #print(user_dict)
    list2=list1[:14]
    # list3=list3.append(list2[0])
    list3=list2[4:14]
    for j in range(len(list3)):
        if j==4:
            #print(list2[j+4])
            just=list2[j+4]
            first=just.split(":")[0]
            second=just.split(":")[1]
            user_dict[first.split("/")[0]]=str(second.split("/")[0]).strip()
            user_dict[first.split("/")[1]]=second.split("/")[1]
        else:
            user_dict[list2[j+4].split(":")[0]]=list2[j+4].split(':')[1].strip()
    user_info.append(user_dict)      
        
    #print(str(list3))
    #print(len(user_dict))
    # for j in range(len(list3)):
    #     user_dict[str(list3).split(":")[0]]=str(list3).split(":")[1]
    # print(user_dict)
    #list3.insert(0,list2[0])
    #print(list3)
    #5가 신장/체중
    #print(list1)
    #user_info.append(list3)
    #1 5 67891011121314
    #break
print(user_info)


100%|████████████████████████████████████████████████████████████████████████████████| 844/844 [06:13<00:00,  2.26it/s]

[{'구단': '한화 이글스', '선수명': '강재민', '등번호': 'No.', '생년월일': '1997년 04월 03일', '포지션': '투수(우언우타)', '신장': '180cm', '체중': '89kg', '경력': '양덕초-마산중-용마고-단국대', '입단 계약금': '8000만원', '연봉': '14500만원', '지명순위': '20 한화 2차 4라운드 38순위', '입단년도': '20한화'}, {'구단': '한화 이글스', '선수명': '권광민', '등번호': 'No.17', '생년월일': '1997년 12월 12일', '포지션': '외야수(좌투좌타)', '신장': '189cm', '체중': '102kg', '경력': '서울청구초-홍은중-장충고', '입단 계약금': '', '연봉': '3300만원', '지명순위': '22 한화 2차 5라운드 41순위', '입단년도': '16시카고컵스'}, {'구단': '한화 이글스', '선수명': '권동욱', '등번호': 'No.02', '생년월일': '2001년 01월 08일', '포지션': '외야수(우투우타)', '신장': '173cm', '체중': '85kg', '경력': '가동초-휘문중-배명고-동국대', '입단 계약금': '', '연봉': '3000만원', '지명순위': '24 한화 육성선수', '입단년도': ''}, {'구단': '한화 이글스', '선수명': '권현', '등번호': 'No.111', '생년월일': '2005년 02월 23일', '포지션': '외야수(우투우타)', '신장': '182cm', '체중': '88kg', '경력': '사당초-언북중-장충고', '입단 계약금': '3000만원', '연봉': '3000만원', '지명순위': '24 한화 10라운드 91순위', '입단년도': ''}, {'구단': '한화 이글스', '선수명': '김강민', '등번호': 'No.9', '생년월일': '1982년 09월 13일', '포지션': '외야수(우투우타)', '신장': '182cm', '체중': '87kg

In [19]:
save_user_info=user_info
print(save_user_info[1])


{'구단': '한화 이글스', '선수명': '권광민', '등번호': 'No.17', '생년월일': '1997년 12월 12일', '포지션': '외야수(좌투좌타)', '신장': '189cm', '체중': '102kg', '경력': '서울청구초-홍은중-장충고', '입단 계약금': '', '연봉': '3300만원', '지명순위': '22 한화 2차 5라운드 41순위', '입단년도': '16시카고컵스'}


In [20]:
import pickle 
# binary save , load 
with open("./kbo.pkl", "wb") as f:
    pickle.dump(playid, f)


In [21]:
import pickle
with open("./kbo.pkl", "rb") as f:
    abc = pickle.load(f)


In [ ]:
# import requests
# play_url="https://www.koreabaseball.com/Record/Player/PitcherDetail/Basic.aspx?playerId={}"
# kbo_r=requests.get(play_url.format
# id값으로 파싱하자
# 교수님꺼
play_url = "https://www.koreabaseball.com/Record/Player/PitcherDetail/Basic.aspx?playerId={}"
from bs4 import BeautifulSoup as BS 
import requests 

def get_kbo(id_):
    kbo_r = requests.get(play_url.format(id_))
    bs = BS(kbo_r.text)
    data = {x.text.split(":")[0] : x.text.split(":")[1] for x in bs.find("div", class_= "player_basic").findAll("li")}
    data['team'] = bs.find("h4", id="h4Team").text
    return data
# get_kbo("53096")

    

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
df=pd.DataFrame([get_kbo(x) for x in playid]) #dataframe만들기

# save_user_info


In [ ]:
!dir

In [22]:
#pip install tqdm
import pickle
with open("./kbo_detail.pkl","rb") as f:
    df=pickle.load(f)

# df1=pd.DataFrame(save_user_info) #dataframe만들기
# import pickle
# with open("./kbo.pkl", "rb") as f:
#     abc = pickle.load(f)

In [321]:
df1=pd.DataFrame(save_user_info) #dataframe만들기

In [23]:
df

,선수명,등번호,생년월일,포지션,신장/체중,경력,입단 계약금,연봉,지명순위,입단년도,team
0,강재민,No.,1997년 04월 03일,투수(우언우타),180cm/89kg,양덕초-마산중-용마고-단국대,8000만원,14500만원,20 한화 2차 4라운드 38순위,20한화,한화 이글스
1,권광민,No.17,1997년 12월 12일,외야수(좌투좌타),189cm/102kg,서울청구초-홍은중-장충고,,3300만원,22 한화 2차 5라운드 41순위,16시카고컵스,한화 이글스
2,권동욱,No.02,2001년 01월 08일,외야수(우투우타),173cm/85kg,가동초-휘문중-배명고-동국대,,3000만원,24 한화 육성선수,,한화 이글스
3,권현,No.111,2005년 02월 23일,외야수(우투우타),182cm/88kg,사당초-언북중-장충고,3000만원,3000만원,24 한화 10라운드 91순위,,한화 이글스
4,김강민,No.9,1982년 09월 13일,외야수(우투우타),182cm/87kg,본리초-대구중-경북고-SK-SSG,11000만원,11000만원,01 SK 2차 2라운드 18순위,01SK,한화 이글스
...,...,...,...,...,...,...,...,...,...,...,...
839,한민우,No.96,1999년 04월 27일,투수(좌투좌타),177cm/82kg,동일중앙초-경남중-경남고,1000만원,3000만원,23 KT 육성선수,,KT 위즈
840,한차현,No.59,1998년 11월 30일,투수(우투우타),180cm/80kg,사능초(남양주리틀)-청원중-포항제철고-성균관대,10000만원,3100만원,21 KT 2차 2라운드 15순위,21KT,KT 위즈
841,홍현빈,No.31,1997년 08월 29일,외야수(우투좌타),174cm/70kg,수원신곡초-매송중-유신고-KT-상무,9000만원,4500만원,17 KT 2차 3라운드 21순위,17KT,KT 위즈
842,황의준,No.103,2002년 04월 06일,외야수(우투좌타),185cm/95kg,칠성초-경복중-경북고-수성대,4000만원,3000만원,23 KT 8라운드 80순위,,KT 위즈


In [25]:
df.shape


(844, 11)

In [26]:
df1.shape

NameError: name 'df1' is not defined

In [27]:
df.duplicated().sum() #중복값 행에서 확인


0

In [28]:
df1.duplicated().sum() #중복값 행에서 확인

NameError: name 'df1' is not defined

In [29]:
df.drop_duplicates().shape #중복값 버리기
# df1.drop_duplicates().shape #중복값 버리기

(844, 11)

In [324]:
# df1.drop_duplicates().shape #중복값 버리기


(844, 12)

In [30]:
df.shape


(844, 11)

In [329]:
# df1.shape

(844, 12)

In [31]:
df.drop_duplicates(inplace=True) #적용해줘
df.shape


(844, 11)

In [ ]:
# df['team'].value_counts()

In [32]:
df['team'].value_counts()

team
한화 이글스      93
NC 다이노스     89
KT 위즈       87
롯데 자이언츠     85
삼성 라이온즈     85
KIA 타이거즈    84
두산 베어스      84
SSG 랜더스     81
LG 트윈스      74
키움 히어로즈     59
고양 히어로즈     23
Name: count, dtype: int64

In [335]:
# df1['team'].value_counts()

KeyError: 'team'

In [33]:
df['포지션'].value_counts()
len(df['포지션'].value_counts().index)


19

In [34]:
df['포지션'].unique().size

19

In [35]:
len(df['포지션'].value_counts())

19

In [36]:
df['신장/체중']

0        180cm/89kg
1       189cm/102kg
2        173cm/85kg
3        182cm/88kg
4        182cm/87kg
           ...     
839      177cm/82kg
840      180cm/80kg
841      174cm/70kg
842      185cm/95kg
843      183cm/96kg
Name: 신장/체중, Length: 844, dtype: object

In [37]:
#magic 명령어 
# %%time
#브로드 캐스팅-파이썬
#벡터연산-numpy
df['신장']=df['신장/체중'].apply(lambda x:int(x.split("/")[0].replace("cm","")))
df['체중']=df['신장/체중'].apply(lambda x:int(x.split("/")[1].replace("kg","")))

In [38]:
df.head(3)

df.tail(3)
df['bmi']=df['체중']/(df['신장']/100)**2
df


,선수명,등번호,생년월일,포지션,신장/체중,경력,입단 계약금,연봉,지명순위,입단년도,team,신장,체중,bmi
0,강재민,No.,1997년 04월 03일,투수(우언우타),180cm/89kg,양덕초-마산중-용마고-단국대,8000만원,14500만원,20 한화 2차 4라운드 38순위,20한화,한화 이글스,180,89,27.469136
1,권광민,No.17,1997년 12월 12일,외야수(좌투좌타),189cm/102kg,서울청구초-홍은중-장충고,,3300만원,22 한화 2차 5라운드 41순위,16시카고컵스,한화 이글스,189,102,28.554632
2,권동욱,No.02,2001년 01월 08일,외야수(우투우타),173cm/85kg,가동초-휘문중-배명고-동국대,,3000만원,24 한화 육성선수,,한화 이글스,173,85,28.400548
3,권현,No.111,2005년 02월 23일,외야수(우투우타),182cm/88kg,사당초-언북중-장충고,3000만원,3000만원,24 한화 10라운드 91순위,,한화 이글스,182,88,26.566840
4,김강민,No.9,1982년 09월 13일,외야수(우투우타),182cm/87kg,본리초-대구중-경북고-SK-SSG,11000만원,11000만원,01 SK 2차 2라운드 18순위,01SK,한화 이글스,182,87,26.264944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,한민우,No.96,1999년 04월 27일,투수(좌투좌타),177cm/82kg,동일중앙초-경남중-경남고,1000만원,3000만원,23 KT 육성선수,,KT 위즈,177,82,26.173833
840,한차현,No.59,1998년 11월 30일,투수(우투우타),180cm/80kg,사능초(남양주리틀)-청원중-포항제철고-성균관대,10000만원,3100만원,21 KT 2차 2라운드 15순위,21KT,KT 위즈,180,80,24.691358
841,홍현빈,No.31,1997년 08월 29일,외야수(우투좌타),174cm/70kg,수원신곡초-매송중-유신고-KT-상무,9000만원,4500만원,17 KT 2차 3라운드 21순위,17KT,KT 위즈,174,70,23.120624
842,황의준,No.103,2002년 04월 06일,외야수(우투좌타),185cm/95kg,칠성초-경복중-경북고-수성대,4000만원,3000만원,23 KT 8라운드 80순위,,KT 위즈,185,95,27.757487


In [39]:
df.sort_values(by=['bmi'],ascending=False)

,선수명,등번호,생년월일,포지션,신장/체중,경력,입단 계약금,연봉,지명순위,입단년도,team,신장,체중,bmi
10,김민우,No.53,1995년 07월 25일,투수(우투우타),186cm/123kg,사파초-마산중-용마고,20000만원,16700만원,15 한화 2차 1라운드 1순위,15한화,한화 이글스,186,123,35.553243
356,김범석,No.55,2004년 05월 21일,포수(우투우타),178cm/110kg,김해삼성초-경남중-경남고,25000만원,3300만원,23 LG 1라운드 7순위,23LG,LG 트윈스,178,110,34.717839
30,박성웅,No.63,1999년 08월 20일,투수(좌투좌타),178cm/109kg,광주서석초-충장중-광주제일고,10000만원,3300만원,18 한화 2차 2라운드 14순위,18한화,한화 이글스,178,109,34.402222
57,이성원,No.98,1999년 11월 02일,내야수(우투우타),185cm/115kg,수원신곡초-매향중-장안고,6000만원,3000만원,18 한화 2차 6라운드 54순위,,한화 이글스,185,115,33.601169
509,김기연,No.45,1997년 09월 07일,포수(우투우타),178cm/106kg,광주수창초-진흥중-진흥고-LG,7000만원,4000만원,16 LG 2차 4라운드 34순위,17LG,두산 베어스,178,106,33.455372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,성지훈,No.97,2000년 01월 29일,투수(좌투좌타),181cm/68kg,송정동초-무등중-광주제일고-동아대,,3000만원,23 한화 육성선수,24한화,한화 이글스,181,68,20.756387
804,엄상백,No.18,1996년 10월 04일,투수(우언우타),187cm/72kg,역삼초-언북중-덕수고-KT-상무,23000만원,25000만원,15 KT 1차,15KT,KT 위즈,187,72,20.589665
821,이태규,No.45,2000년 02월 21일,투수(우투좌타),188cm/72kg,희망대초-매향중-장안고-KIA,8000만원,3000만원,19 KIA 2차 3라운드 30순위,23KIA,KT 위즈,188,72,20.371209
379,백선기,No.114,1998년 08월 27일,투수(좌투좌타),186cm/70kg,남도초-경상중-대구상원고-KT,5000만원,3000만원,18 KT 2차 7라운드 61순위,,LG 트윈스,186,70,20.233553


In [40]:
df.drop(['신장/체중'],axis=1, inplace=True) #axis=1은 컬럼 리스트형식이다
df

,선수명,등번호,생년월일,포지션,경력,입단 계약금,연봉,지명순위,입단년도,team,신장,체중,bmi
0,강재민,No.,1997년 04월 03일,투수(우언우타),양덕초-마산중-용마고-단국대,8000만원,14500만원,20 한화 2차 4라운드 38순위,20한화,한화 이글스,180,89,27.469136
1,권광민,No.17,1997년 12월 12일,외야수(좌투좌타),서울청구초-홍은중-장충고,,3300만원,22 한화 2차 5라운드 41순위,16시카고컵스,한화 이글스,189,102,28.554632
2,권동욱,No.02,2001년 01월 08일,외야수(우투우타),가동초-휘문중-배명고-동국대,,3000만원,24 한화 육성선수,,한화 이글스,173,85,28.400548
3,권현,No.111,2005년 02월 23일,외야수(우투우타),사당초-언북중-장충고,3000만원,3000만원,24 한화 10라운드 91순위,,한화 이글스,182,88,26.566840
4,김강민,No.9,1982년 09월 13일,외야수(우투우타),본리초-대구중-경북고-SK-SSG,11000만원,11000만원,01 SK 2차 2라운드 18순위,01SK,한화 이글스,182,87,26.264944
...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,한민우,No.96,1999년 04월 27일,투수(좌투좌타),동일중앙초-경남중-경남고,1000만원,3000만원,23 KT 육성선수,,KT 위즈,177,82,26.173833
840,한차현,No.59,1998년 11월 30일,투수(우투우타),사능초(남양주리틀)-청원중-포항제철고-성균관대,10000만원,3100만원,21 KT 2차 2라운드 15순위,21KT,KT 위즈,180,80,24.691358
841,홍현빈,No.31,1997년 08월 29일,외야수(우투좌타),수원신곡초-매송중-유신고-KT-상무,9000만원,4500만원,17 KT 2차 3라운드 21순위,17KT,KT 위즈,174,70,23.120624
842,황의준,No.103,2002년 04월 06일,외야수(우투좌타),칠성초-경복중-경북고-수성대,4000만원,3000만원,23 KT 8라운드 80순위,,KT 위즈,185,95,27.757487


In [44]:
df.groupby(['team'])['bmi'].mean() #datatype series

team
KIA 타이거즈    26.544495
KT 위즈       25.736477
LG 트윈스      26.030905
NC 다이노스     25.957081
SSG 랜더스     25.967989
고양 히어로즈     25.514269
두산 베어스      25.987765
롯데 자이언츠     26.481472
삼성 라이온즈     26.194392
키움 히어로즈     26.615679
한화 이글스      26.665788
Name: bmi, dtype: float64

In [43]:
df.groupby(['team']),as_index=False)['bmi'].mean() #datatype series


SyntaxError: cannot assign to function call (4286962741.py, line 1)

In [352]:
sum(range(1,11))/10


5.5

In [46]:
# save_user_info
# dt
import pandas as pd
import numpy as np
np.array(np.arange(1,11)).std()
pd.DataFrame({'a',range(1,11)}).std()
#자유도


TypeError: could not convert string to float: 'a'